## Read Dataset

In [2]:
import pandas as pd
import re

In [3]:
send_dataset = pd.read_csv('dataset2_send_pass.csv', compression='gzip')
send_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   address           108 non-null    object
 1   pass              108 non-null    bool  
 2   creation_date     108 non-null    object
 3   code              108 non-null    object
 4   contract_name     108 non-null    object
 5   compiler_version  108 non-null    object
 6   types             108 non-null    object
 7   hash_code         108 non-null    object
dtypes: bool(1), object(7)
memory usage: 6.1+ KB


In [4]:
send_dataset.sample(3)

,address,pass,creation_date,code,contract_name,compiler_version,types,hash_code
13,0x6f9073c5c718af1791d1b8c1cc90f47a8c14cf27,True,2020-08-25 20:43:55 UTC,/**\n * @authors: [@mtsalenc]\n * @reviewers...,GeneralizedTCR,0.5.17,send,92bf7443ea9b07b46605c7c7e88697a72dddfc70168859...
41,0xad412acc218777bc129871f37bb2b278c48f085f,True,2020-03-09 15:16:20 UTC,pragma solidity ^0.5.16;\n\nlibrary SafeMath {...,Dex,0.5.16,send,a309728729743d1b1936d84e648c13cbd004bece46b7c4...
73,0x3a7edc1eb16cb454e4964ec51642f7d7b35b5292,True,2020-08-25 00:10:27 UTC,pragma solidity ^0.4.26;\n\nlibrary CappedMath...,KlerosGovernor,0.4.26,send,98af1e3b5db3e891adc514e7ef7cc8b62723a3955542dc...


In [5]:
call_dataset = pd.read_csv('dataset2_call_conststring_pass.csv', compression='gzip')
call_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346 entries, 0 to 345
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   address           346 non-null    object
 1   pass              346 non-null    bool  
 2   creation_date     346 non-null    object
 3   code              346 non-null    object
 4   contract_name     346 non-null    object
 5   compiler_version  346 non-null    object
 6   types             346 non-null    object
 7   hash_code         346 non-null    object
dtypes: bool(1), object(7)
memory usage: 19.4+ KB


## Replace Send Patterns

In [6]:
def rreplace(string, ori, new, times):
    return new.join(string.rsplit(ori, times))


def process_line(line):
    #         if (!owner.send(amount)) revert();
    pattern_count = [0, 0, 0, 0, 0] # if, ifnot, return, require, bare
    
    if re.search('if *\(!*.*\.send *\(.*\).*\) *revert\(\);', line):
        string_before = re.search('if *\(!*.*\.send *\(.*\).*\) *revert\(\);', line).group()
        string_after = string_before.replace('if', '').replace('!', '').replace('.send(', '.transfer(').replace('.send (', '.transfer(')
        string_after = string_after.replace('revert();', '')
        if '==' in string_after:
            string_after = string_after.split('==')[0]
        string_after = string_after.strip('() ') + ');'

        line = line.replace(string_before, string_after)
        pattern_count[1] += 1

    # if (!addr.send(some_ether)) {
    elif re.search('if *\(![a-zA-Z_\.\(\)_\[\]]*\.send *\(.*\) *{*', line):
        string_before = re.search('if *\(![a-zA-Z_\.\(\)_\[\]]*\.send *\(.*\) *{*', line).group()
        string_after = string_before.replace('if', '').replace('!', '').replace('.send(', '.transfer(').replace('.send (', '.transfer(')
        if '{' in string_after:
            #string_after = string_after.strip('(){ ') + ');'
            string_after = string_after.replace('(', '', 1).replace('{', '', 1)
            string_after = rreplace(string_after, ')', '', 1) + ';'
            string_after += '\nif(false)\n{'
        else:
            string_after = string_after.strip('() ') + ');'
            string_after += '\nif(false)'

        line = line.replace(string_before, string_after)
        pattern_count[0] += 1

    # if (addr.send(some_ether)) {
    elif re.search('if *\([a-zA-Z_\.\(\)_\[\]]*\.send *\(.*\) *{*', line):
        string_before = re.search('if *\([a-zA-Z_\.\(\)_\[\]]*\.send *\(.*\) *{*', line).group()
        string_after = string_before.replace('if', '').replace('.send(', '.transfer(').replace('.send (', '.transfer(')
        if '==' in string_after:
            if '{' in string_after:
                string_after = string_after.split('==')[0] + ') {'
                string_after = string_after.replace('(', '', 1).replace('{', '', 1)
                string_after = rreplace(string_after, ')', '', 1) + ';'
                string_after += '\nif(true)\n{'
            else:
                string_after = string_after.strip('() ') + ');'
                string_after += '\nif(true)'
        else:
            if '{' in string_after:
                string_after = string_after.replace('(', '', 1).replace('{', '', 1)
                string_after = rreplace(string_after, ')', '', 1) + ';'
                string_after += '\nif(true)\n{'
            else:
                string_after = string_after.strip('() ') + ');'
                string_after += '\nif(true)'
        line = line.replace(string_before, string_after)
        pattern_count[1] += 1

    # return msg.sender.send(
    elif re.search('return .*\.send *\(', line):
        string_before = re.search('return .*\.send *\(', line).group()
        string_after = string_before.replace('return', '').replace('.send(', '.transfer(').replace('.send (', '.transfer(')
        line = line.replace(string_before, string_after)
        pattern_count[2] += 1

    # require(any.send(some_ether), 'message');
    elif re.search('require *\([^!,]*\.send\([^\,]*\), *[\d\w_ \"\'\.]*\);', line):
        string_before = re.search('require *\([^!,]*\.send\([^\,]*\), *[\d\w_ \"\'\.]*\);', line).group()
        string_after = string_before.replace('require(', '').replace('require (', '').replace('.send(', '.transfer(').replace('.send (', '.transfer(')
        if string_after.find(',') != -1:
            string_after = string_after[:string_after.find(',')]
            string_after += ';'
        else:
            string_after = rreplace(string_after, ')', '', 1)
        line = line.replace(string_before, string_after)
        pattern_count[3] += 1

    # require(any.send(some_ether));
    elif re.search('require *\([^!,]*\.send\([^\,]*\)\);', line):
        string_before = re.search('require *\([^!,]*\.send\([^\,]*\)\);', line).group()
        string_after = string_before.replace('require(', '').replace('require (', '').replace('.send(', '.transfer(').replace('.send (', '.transfer(')
        if string_after.find(',') != -1:
            string_after = string_after[:string_after.find(',')]
            string_after += ';'
        else:
            string_after = rreplace(string_after, ')', '', 1)
        line = line.replace(string_before, string_after)
        pattern_count[3] += 1

    # assert(any.send(some_ether));
    elif re.search('assert\([^!,]*\.send\(.*\)\);', line):
        string_before = re.search('assert\([^!,]*\.send\(.*\)\);', line).group()
        string_after = string_before.replace('assert(', '').replace('.send(', '.transfer(').replace('.send (', '.transfer(')
        if string_after.find(',') != -1:
            string_after = string_after[:string_after.find(',')]
            string_after += ';'
        else:
            string_after = rreplace(string_after, ')', '', 1)
        line = line.replace(string_before, string_after)
        pattern_count[3] += 1

    #bool re_s = address(this).send(any_ether);
    elif re.search(' *= *[a-zA-Z_\.\(\)_\[\]]*\.send\([a-zA-Z_.\(\)]*\);', line):
        string_before = re.search(' *= *[a-zA-Z_\.\(\)_\[\]]*\.send\([a-zA-Z_.\(\)]*\);', line).group()
        string_after = string_before.replace('=', '= true;\n').replace('.send(', '.transfer(').replace('.send (', '.transfer(')

        line = line.replace(string_before, string_after)
        pattern_count[4] += 1
    
    # any.send(Any) No any.send(Any, b, c)
    elif re.search('\.send *\([^\,\n]*\)', line):
        string_before = re.search('\.send *\([^\,\n]*\)', line).group()
        string_after = string_before.replace('.send(', '.transfer(').replace('.send (', '.transfer(')
        line = line.replace(string_before, string_after)
        pattern_count[4] += 1

    return line, pattern_count

In [7]:
line = 'if (!address(uint160(pool)).send(poolAmount)) {'
line = 'return super.send(_to, _value);'
print(process_line(line))

(' super.transfer(_to, _value);', [0, 0, 1, 0, 0])


In [8]:
"""
import shutil

dest_dir_path = '/tmp/generated'

if os.path.exists(dest_dir_path):
    shutil.rmtree(dest_dir_path)
if not os.path.exists(dest_dir_path):
    os.makedirs(dest_dir_path)
"""

"\nimport shutil\n\ndest_dir_path = '/tmp/generated'\n\nif os.path.exists(dest_dir_path):\n    shutil.rmtree(dest_dir_path)\nif not os.path.exists(dest_dir_path):\n    os.makedirs(dest_dir_path)\n"

In [9]:
dest_dir_path = 'contract_send_replaced'

total_stats = [0, 0, 0, 0, 0] # if, ifnot, return, require, bare
replaced_counter = 0
for _, r in send_dataset.iterrows():
    replaced_flag = False
    code = r['code']
    addr = r['address']
    newlines = []
    contract_stats, stats = [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]
    
    for line in code.split('\n'):
        
        if re.search('\.send *\(', line):
            line, stats = process_line(line)
            replaced_flag = True
        newlines.append(line)
        contract_stats = [1 if sum(x) else 0 for x in zip(stats, contract_stats)]
        
    
    #contract_stats = [0, 0, 0, 0, 0, 1] if sum(contract_stats) > 1 else contract_stats + [0]

    total_stats = [sum(x) for x in zip(contract_stats, total_stats)]
    
    if replaced_flag:
        replaced_counter+=1
        
replaced_counter
            

108

In [10]:
total_stats

[15, 13, 3, 15, 64]

In [47]:
['{:.2f}'.format(i/sum(total_stats)) for i in total_stats]

['0.12', '0.08', '0.03', '0.14', '0.58', '0.06']

## Replace Call Pattern

In [11]:
from string import ascii_lowercase
import re

pattern_sha3 = '[\w\d_\(\)]+\.call\(bytes4\((bytes32\()*(sha3|keccak256)\("[\w\d_]+\([\w\d_,]*\)"\)+(, [\w\d_,. \[\]]*\)+)*'
pattern_abi = '[\w\d_\(\)]+\.call\(abi\.encodeWithSignature\("[\w\d_]+\([\w\d_,]*\)"\)*(, [\w\d_,. ]*\)+)*'

"""
require(_spender.call(bytes4(bytes32(keccak256("receiveApproval(address,uint256,address,bytes)"))), msg.sender, _value, this, _extraData));
bool callOk = address(to).call(bytes4(keccak256("onKSCReceived(address,address,uint256,uint8)")), owner, spender, value, receiveType);
require(_token.call(bytes4(keccak256("symbol()"))));
[\w\d_\(\)]+\.call\(bytes4\((bytes32\()*(sha3|keccak256)\("[\w\d_]+\([\w\d_,]*\)"\)+(, [\w\d_,. ]*\)+)*

return receiver.call(abi.encodeWithSignature("onTokenTransfer(address,uint256,bytes)", from, value, data));
require(address(proxy).call(abi.encodeWithSignature("init(bytes)", _data)), "Init failed.");
[\w\d_\(\)]+\.call\(abi\.encodeWithSignature\("[\w\d_]+\([\w\d_,]*\)"\)*(, [\w\d_,. ]*\)+)*
"""

def extract_string(pattern, string):
    return re.search(pattern, string).group()

def extract_dest_address(string):
    dest_addr = string.split('.')[0]
    return ''.join(dest_addr.split('require('))

def extract_function_name(string):
    return string.split('"')[1]

def extract_paramter_list(string):
    return string.split('"')[2].strip('(), ')

def reconstruct_function_call(string):
    da = extract_dest_address(string)
    fn = extract_function_name(string).split('(')[0]
    pl = extract_paramter_list(string)
    return f'test({da}).{fn}({pl});\n'

def reconstruct_function_declearation(string):
    fn = extract_function_name(string)
    pl = fn.strip(')').split('(')[-1]
    alphabet = list(ascii_lowercase)
    pl_add_var_name = ', '.join([a + ' ' + b for (a,b) in zip(pl.split(','), alphabet)]) if pl else ''
    fn_add_var_name = fn.split('(')[0] + '(' + pl_add_var_name + ')'
    return f'function {fn_add_var_name};'


In [12]:
def rreplace(string, ori, new, times):
    return new.join(string.rsplit(ori, times))

string = 'require(ethFundDeposit.send(address(this).balance));'
print(rreplace(string, ')', '', 1))

require(ethFundDeposit.send(address(this).balance);


In [13]:
def process_line(line):
    
    pattern_sha3 = '[\w\d_\(\)]+\.call\(bytes4\((bytes32\()*(sha3|keccak256)\("[\w\d_]+\([\w\d_,]*\)"\)+(, [\w\d_,. \[\]]*\)+)*'
    pattern_abi = '[\w\d_\(\)]+\.call\(abi\.encodeWithSignature\("[\w\d_]+\([\w\d_,]*\)"\)*(, [\w\d_,. ]*\)+)*'
    func_decl = None
    
    if re.search(pattern_sha3, line):
        func_call = reconstruct_function_call(extract_string(pattern_sha3, line))
        func_decl = reconstruct_function_declearation(extract_string(pattern_sha3, line))
        line = line.replace(line, func_call)

    elif re.search(pattern_abi, line):
        func_call = reconstruct_function_call(extract_string(pattern_abi, line))
        func_decl = reconstruct_function_declearation(extract_string(pattern_abi, line))
        line = line.replace(line, func_call)

    return line, func_decl

In [ ]:
string = 'bool callOk = address(to).call(bytes4(keccak256("onKSCReceived(address,address,uint256,uint8)")), owner, spender, value, receiveType);'
print(reconstruct_function_call(extract_string(pattern_sha3, string)))
print(reconstruct_function_declearation(extract_string(pattern_sha3, string)))

string = 'require(address(proxy).call(abi.encodeWithSignature("init(bytes)", _data)), "Init failed.");'
print(reconstruct_function_call(extract_string(pattern_abi, string)))
print(reconstruct_function_declearation(extract_string(pattern_abi, string)))

string = 'return receiver.call(abi.encodeWithSignature("onTokenTransfer(address,uint256,bytes)", from, value, data));'
print(reconstruct_function_call(extract_string(pattern_abi, string)))
print(reconstruct_function_declearation(extract_string(pattern_abi, string)))

string = 'require(_token.call(bytes4(keccak256("symbol()"))));'
print(reconstruct_function_call(extract_string(pattern_sha3, string)))
print(reconstruct_function_declearation(extract_string(pattern_sha3, string)))

string = 'if(!ESSgenesis.call(bytes4(keccak256("transfer(address,uint256)")), msg.sender, sold[msg.sender])){revert();}'
print(reconstruct_function_call(extract_string(pattern_sha3, string)))
print(reconstruct_function_declearation(extract_string(pattern_sha3, string)))

In [ ]:
dest_dir_path = 'contract_call_replaced'

for _, r in call_dataset.iterrows():
    code = r['code']
    addr = r['address']
    newlines = []
    func_decls = []

    for line in code.split('\n'):
        func_decl = None
        if re.search('\.call *\(', line):
            line, func_decl = process_line(line)
            #print(line)
        newlines.append(line)
        if func_decl:
            func_decls.append(func_decl)
        
    with open(f'../{dest_dir_path}/{addr}.sol', 'w') as f:
        f.write('\n'.join(newlines))
        f.write('\ncontract test {\n\t')
        f.write('\n\t'.join([x for x in func_decls]))
        f.write('\n}')